# Neo4j Graph Store

In [29]:
%pip install -q llama-index-llms-openai
%pip install -q llama-index-graph-stores-neo4j
%pip install -q llama-index-embeddings-openai
#%pip install -q llama-index-llms-azure-openai


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


https://docs.llamaindex.ai/en/stable/examples/index_structs/knowledge_graph/Neo4jKGIndexDemo/

In [ ]:
#!pip uninstall -y llama-index-readers-wikipedia

In [2]:

# For OpenAI

import os

os.environ["OPENAI_API_KEY"] = ""

import logging
import sys
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

logging.basicConfig(stream=sys.stdout, level=logging.INFO)

# define LLM
llm = OpenAI(temperature=0, model="gpt-4o")
Settings.llm = llm
Settings.chunk_size = 512

In [3]:
from llama_index.core import KnowledgeGraphIndex, SimpleDirectoryReader
from llama_index.core import StorageContext
from llama_index.graph_stores.neo4j import Neo4jGraphStore


from llama_index.llms.openai import OpenAI
from IPython.display import Markdown, display

In [4]:
username = "neo4j"
password = "12345678"
url = "bolt://localhost:7687"
database = "neo4j"

In [8]:
!python3 -m pip install -U llama-index-readers-file

  Using cached llama_index_readers_file-0.4.3-py3-none-any.whl.metadata (5.4 kB)
  Using cached pypdf-5.1.0-py3-none-any.whl.metadata (7.2 kB)
  Using cached striprtf-0.0.26-py3-none-any.whl.metadata (2.1 kB)
Using cached llama_index_readers_file-0.4.3-py3-none-any.whl (38 kB)
Using cached pypdf-5.1.0-py3-none-any.whl (297 kB)
Using cached striprtf-0.0.26-py3-none-any.whl (6.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 2.9 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [9]:
from llama_index.core.llama_dataset import download_llama_dataset

In [10]:
#from llama_index.core import SimpleDirectoryReader
#from llama_index.core.readers.file.base import SimpleDirectoryReader
reader = SimpleDirectoryReader(
    input_files=["./paul_graham/source_files/source.txt"]
)
docs = reader.load_data()
print(f"Loaded {len(docs)} docs")

Loaded 1 docs


In [11]:
graph_store = Neo4jGraphStore(
    username=username,
    password=password,
    url=url,
    database=database,
)

storage_context = StorageContext.from_defaults(graph_store=graph_store)

# NOTE: can take a while!
index = KnowledgeGraphIndex.from_documents(
    docs,
    storage_context=storage_context,
    max_triplets_per_chunk=2,
)

INFO:neo4j.notifications:Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Schema.IndexOrConstraintAlreadyExists} {category: SCHEMA} {title: `CREATE CONSTRAINT IF NOT EXISTS FOR (e:Entity) REQUIRE (e.id) IS UNIQUE` has no effect.} {description: `CONSTRAINT constraint_1ed05907 FOR (e:Entity) REQUIRE (e.id) IS UNIQUE` already exists.} {position: None} for query: '\n                CREATE CONSTRAINT IF NOT EXISTS FOR (n:Entity) REQUIRE n.id IS UNIQUE;\n                '
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST

In [19]:
query_engine = index.as_query_engine(
    include_text=False, response_mode="tree_summarize"
)

response = query_engine.query("What did author write")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [20]:
response

Response(response='The author wrote "Hackers & Painters."', source_nodes=[NodeWithScore(node=TextNode(id_='6e6e7c62-3428-4e6f-a20c-07f4bf188ee7', embedding=None, metadata={'kg_rel_texts': ["['CALLED', 'Hackers & painters']"], 'kg_rel_map': {'Book': [['CALLED', 'Hackers & painters']]}, 'kg_schema': {'schema': 'Node properties are the following:\n\nRelationship properties are the following:\n\nThe relationships are the following:\n'}}, excluded_embed_metadata_keys=['kg_rel_map', 'kg_rel_texts'], excluded_llm_metadata_keys=['kg_rel_map', 'kg_rel_texts'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text="The following are knowledge sequence in max depth 2 in the form of directed graph like:\n`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...`\n['CALLED', 'Hackers & painters']", mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'), score=1000.0)], met

In [21]:
display(Markdown(f"<b>{response}</b>"))

<b>The author wrote "Hackers & Painters."</b>

In [22]:

query_engine = index.as_query_engine(
    include_text=True, response_mode="tree_summarize"
)
response = query_engine.query(
    "Whom did Paul Graham Visit?"
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [24]:
display(Markdown(f"<b>{response}</b>"))

<b>Paul Graham visited Rich Draves.</b>

In [26]:
# Clean dataset first
graph_store.query(
    """
MATCH (n) DETACH DELETE n
"""
)

# NOTE: can take a while!
index = KnowledgeGraphIndex.from_documents(
    docs,
    storage_context=storage_context,
    max_triplets_per_chunk=2,
    include_embeddings=True,
)

query_engine = index.as_query_engine(
    include_text=True,
    response_mode="tree_summarize",
    embedding_mode="hybrid",
    similarity_top_k=5,
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST

In [27]:
#query_engine = index.as_query_engine(
#    include_text=False, response_mode="tree_summarize"
#)

response = query_engine.query("What did author write")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: e5962aaa-dadc-48de-8509-10ba906c490f: This was when I really started programming. I wrote simple games, a program t...
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 426cc155-3024-4fad-bf21-b90d30916731: Occasionally after wrestling for hours with some gruesome bug I'd check Twitt...
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 1bd64995-d210-42f1-aad9-704c7c54e2ac: So with my unerring nose for financial opportunity, I decided to write anothe...
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: a9666940-fff0-4caa-96c3-24c0b34399d5: So I decided to take a shot at it. It took 4 years, from March 26, 2015 to Oc...
INFO:llama_index.core.indices.knowle

In [28]:
display(Markdown(f"<b>{response}</b>"))

<b>The author wrote simple games, a word processor, a book about Lisp hacking, a book on Lisp, and Bel.</b>